## Julia
### "looks like python, feels like lisp, runs like C"
Resources for presentation: https://youtu.be/8h8rQyEpiZA https://youtu.be/EkgCENBFrAY

Python diffs: https://docs.julialang.org/en/v1/manual/noteworthy-differences/#Noteworthy-differences-from-Python

Resources for learning: https://benlauwens.github.io/ThinkJulia.jl/latest/book.html https://ucidatascienceinitiative.github.io/IntroToJulia/

In [ ]:
#Imports
using RDatasets, Clustering, DataFrames, PyCall, Conda, BenchmarkTools, Libdl, Plots

### Basics
#### Printing
Use ```println()```. Julia uses ```*``` for string concatonation.
You can also use string templates by passing Julia code into ```$()``` in a string
#### Strings
Strings need to be enclosed in "" or """ """. '' is for chars

In [ ]:
temp = "like this!"
println("Julia Basics!\n")
println("String concatonation uses " * "a * rather than +...weird, I know")
println("Or you can pass it, ", temp)
println("Template strings work $temp\n")
println("Type of temp: $(typeof(temp))")

#### Arithmetic

In [ ]:
num1 = 3
num2 = 4
# By the way this is a comment #
#=
This is a lonnnnnnnnnnnnnnnnnnnnnnng comment
=#
println("Num1: $num1, Num2: $num2. Num types: $(typeof(num1))\n")
println("Sum: $(num1 + num2)")
println("Difference: $(num1 - num2)")
println("Multiplication: $(num1 * num2)")
println("Division $(num1/num2)")
println("Modulus $(num2 % num1)")
println("Exponentiate (num1^2): $(num1^2)")
println("Integer divide: $(num1 ÷ num2)")

In [ ]:
"hi " ^ 10 # repeating strings

#### Data Structures
Indexed at 1 :(

In [ ]:
my_tuple = ("first", "second", "third")
my_tuple[1]

In [ ]:
my_tuple = (el_one="first", el_two="second", el_three="third")
my_tuple.el_two == my_tuple[2]

In [ ]:
mydict = Dict("name" => "Saul Goodman", "age" => 40)

In [ ]:
mydict["occupation"] = "lawyer"
mydict

function calls that end with a bang(!) indicate they mutate in place

In [ ]:
pop!(mydict, "occupation")

In [ ]:
mydict

#### Loops

In [ ]:
myarr = []
for i in 1:5  # also can do steps 1:2:10
    push!(myarr, i)
end
myarr

In [ ]:
for i in myarr
    println(i)
end

In [ ]:
m,n = 4,4
A = fill(0, (m,n))

In [ ]:
# list comprehension also works, same as python
for i in 1:m, j in 1:n
    A[i,j] = i + j
end
A

### Linear Algebra

In [ ]:
A = rand(3,3)

In [ ]:
A'A

#### Conditionals

In [ ]:
if false
    print("if\n")
elseif true
    print("elseif!\n")
else
    print("else!\n")
end
println(true ? "true!" : "false!")

#### Functions

In [ ]:
function hello1(name)
    println("Hello $name\n")
end

hello2(name) = println("Hello again, $name\n")

In [ ]:
hello1("Ashik")
hello2("Ashik")

In [ ]:
# Anonymous functions
square = x -> x^2
square(5)

### Now for more interesting features!
#### Mutating/Non Mutating functions

In [ ]:
arr = [5,2,1,3,4]
test= sort(arr)    # add a bang for mutation in place ! -> sort!(arr)
print("arr: $arr")
print('\n')
print("test: $test")

### Broadcasting

In [ ]:
A = [1:3 4:6 7:9]

In [ ]:
square(A) # matrix multiplication: A*A

In [ ]:
square.(A) # square every element of A

### Fun stuff
##### Unicode: https://docs.julialang.org/en/v1/manual/unicode-input/

In [ ]:
🍎 = 5;
α = 6;
🐢 = 1:10;

In [ ]:
🍎 ≤ α

In [ ]:
if 🍎 ∈ 🐢
    println("The apple is in the turtle!")
end

#### Macros

In [ ]:
b = 5
c = 6
@code_native b + c

#### Write your own
```
macro show(expression)
  quote
    value = $expression
    println($(Meta.quot(expression)), " = ", value)
    value
  end
end
```

In [ ]:
@show 1+1

In [ ]:
mini_exp = :(b+4)

In [ ]:
eval(mini_exp)

### Plots

In [ ]:
temperatures = [14.4, 14.5, 14.8, 15.2, 15.5, 15.8];
numpirates = [45000, 20000, 15000, 5000, 400, 17];

In [ ]:
gr() # call backend
# plotlyjs()

In [ ]:
plot(numpirates, temperatures, label="line")
scatter!(numpirates, temperatures, label="points") # bang to addend to last plot, rather than make two

### Benchmarking

In [ ]:
times = Dict()

In [ ]:
a = rand(10^7); #10 mil element array

In [ ]:
C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname() # make a temp file

#compile
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code)
end

# define Julia funciton that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

In [ ]:
c_sum(a)
c_sum(a) ≈ sum(a)

In [ ]:
# Conda.add("numpy")

In [ ]:
pysum = pybuiltin("sum")
py_numpy_sum = pyimport("numpy")["sum"]

In [ ]:
pysum(a) ≈ sum(a)

In [ ]:
py_numpy_sum(a) ≈ sum(a)

In [ ]:
c_bench = @benchmark c_sum(a);
jl_bench = @benchmark sum(a);
py_bench = @benchmark pysum(a);
py_numpy = @benchmark py_numpy_sum(a);

In [ ]:
times["C"] = minimum(c_bench.times)/1e6
times["jl"] = minimum(jl_bench.times)/1e6
times["py"] = minimum(py_bench.times)/1e6
times["py_numpy"] = minimum(py_numpy.times)/1e6
times

### Multiprocessing

In [ ]:
function simd_sum(A::Array{Float64,1})
    s = 0.0
    @simd for a in A
        s += a
    end
    s
end #more to show in REPL

In [ ]:
simd_sum(a)

In [ ]:
@which 4.0 + 5.0

### Machine Learning

In [ ]:
iris = dataset("datasets", "iris");

In [ ]:
features = collect(Matrix(iris[:,1:4])')

In [ ]:
result = kmeans(features, 4);

In [ ]:
scatter(iris.PetalLength, iris.PetalWidth, 
        marker_z = result.assignments, 
        color =:blue, legend = false)

In [ ]:
counts(result)
assignments(result)